In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

noise_level_path = './2017_poblacio_exposada_barris_mapa_estrategic_soroll_bcn_long.csv'
rent_path = './2017_lloguer_preu_trim.csv'

rent = pd.read_csv(rent_path)
noise_level = pd.read_csv(noise_level_path)

In [3]:
df_pivoted = rent.pivot_table(index=['Any', 'Trimestre', 'Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri'],
                              columns='Lloguer_mitja',
                              values='Preu',
                              aggfunc='first').reset_index()

# Renombrar las columnas
df_pivoted.rename(columns={'Lloguer mitjà mensual (Euros/mes)': 'Precio_Mensual',
                          'Lloguer mitjà per superfície (Euros/m2 mes)': 'Precio_Por_Superficie'},
                 inplace=True)

# Si deseas combinar las dos columnas en un solo DataFrame, puedes hacerlo así
df_combined = df_pivoted[['Any', 'Trimestre', 'Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri', 'Precio_Mensual', 'Precio_Por_Superficie']]

rent = df_combined

In [4]:
# Computing middle value for Month price and m2 price

price_month=df_pivoted.groupby('Codi_Barri')['Precio_Mensual'].mean()
price_m2=df_pivoted.groupby('Codi_Barri')['Precio_Por_Superficie'].mean()

In [5]:
# Agrupar por distrito y barrio y calcular el promedio de Precio_Por_Superficie
rent_middle = df_pivoted.groupby(['Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri']).agg(
    Precio_Por_Superficie_Mean=('Precio_Por_Superficie', 'mean'),
    Precio_Mensual_Mean=('Precio_Mensual', 'mean')
).reset_index()

In [6]:
df_joined = rent_middle.merge(noise_level, on = ['Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri'])
to_drop = ['Codi_Districte', 'Codi_Barri']
for drop in to_drop:
    df_joined=df_joined.drop(drop,axis=1)

In [7]:
def transform_value(x):
    return np.float16(x.replace("%",""))/100
    
df_joined['Valor']=df_joined['Valor'].apply(transform_value)

df = df_joined.copy()

In [8]:
mapeo_rango_db = {
    '<40 dB': 30,
    '40-45 dB': 42.5,
    '45-50 dB': 47.5,
    '50-55 dB': 52.5,
    '55-60 dB': 57.5,
    '60-65 dB': 62.5,
    '65-70 dB': 67.5,
    '70-75 dB': 72.5,
    '75-80 dB': 77.5,
    '>=80 dB': 90 
}
df['Valor_Numerico'] = df['Rang_soroll'].map(mapeo_rango_db)
df['dB_Promig'] = df['Valor_Numerico'] * df['Valor']
df_dB = df.groupby(['Nom_Districte', 'Nom_Barri', 'Precio_Por_Superficie_Mean', 'Precio_Mensual_Mean', 'Concepte'])['dB_Promig'].sum().reset_index()

dum=pd.get_dummies(df_dB, columns=['Concepte'],prefix=['Concepte'])

In [9]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

#Extract columns from characteristics
X = dum.drop(['Nom_Districte', 'Nom_Barri'], axis=1)

# Standarized values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
pca = PCA(n_components=2)  
pca_result = pca.fit_transform(X)
y=[str(nbh) for nbh in dum['Nom_Barri']]
print(dum.columns)
concept_cols = ['Concepte_FFCC_D',
       'Concepte_FFCC_DEN', 'Concepte_FFCC_E', 'Concepte_FFCC_N',
       'Concepte_GI_TR_D', 'Concepte_GI_TR_DEN', 'Concepte_GI_TR_E',
       'Concepte_GI_TR_N', 'Concepte_INDUST_D', 'Concepte_INDUST_DEN',
       'Concepte_INDUST_E', 'Concepte_INDUST_N', 'Concepte_OCI_N',
       'Concepte_PARCS_D', 'Concepte_PATIS_D', 'Concepte_PATIS_E',
       'Concepte_TOTAL_D', 'Concepte_TOTAL_DEN', 'Concepte_TOTAL_E',
       'Concepte_TOTAL_N', 'Concepte_TRANSIT_D', 'Concepte_TRANSIT_DEN',
       'Concepte_TRANSIT_E', 'Concepte_TRANSIT_N', 'Concepte_VIANANTS_D',
       'Concepte_VIANANTS_E']
y_district = [col for col in concept_cols if dum[col].values == 1]
y_district_info = list(zip(dum['Nom_Districte'], dum['Concepte']))

# Combinar las dos listas para obtener una lista de tuplas
y_district= [(dist, concept) + (col,) for dist, concept, col in zip(y_district_info, y_district)]

Index(['Nom_Districte', 'Nom_Barri', 'Precio_Por_Superficie_Mean',
       'Precio_Mensual_Mean', 'dB_Promig', 'Concepte_FFCC_D',
       'Concepte_FFCC_DEN', 'Concepte_FFCC_E', 'Concepte_FFCC_N',
       'Concepte_GI_TR_D', 'Concepte_GI_TR_DEN', 'Concepte_GI_TR_E',
       'Concepte_GI_TR_N', 'Concepte_INDUST_D', 'Concepte_INDUST_DEN',
       'Concepte_INDUST_E', 'Concepte_INDUST_N', 'Concepte_OCI_N',
       'Concepte_PARCS_D', 'Concepte_PATIS_D', 'Concepte_PATIS_E',
       'Concepte_TOTAL_D', 'Concepte_TOTAL_DEN', 'Concepte_TOTAL_E',
       'Concepte_TOTAL_N', 'Concepte_TRANSIT_D', 'Concepte_TRANSIT_DEN',
       'Concepte_TRANSIT_E', 'Concepte_TRANSIT_N', 'Concepte_VIANANTS_D',
       'Concepte_VIANANTS_E'],
      dtype='object')


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:

import matplotlib.pyplot as plt
import mplcursors  # Importa la biblioteca mplcursors

# Supongamos que tienes un DataFrame con los componentes principales en 'pca_result' y los nombres de los barrios en 'y'


# Crear un diccionario para asignar colores a cada barrio
colores = {}
barrios_unicos = set(y_district)
colormap = plt.cm.get_cmap('rainbow', len(barrios_unicos))
for i, barrio in enumerate(barrios_unicos):
    colores[barrio] = colormap(i / len(barrios_unicos))

# Crear un gráfico de dispersión
fig, ax = plt.subplots(figsize=(14, 10))

scatter = ax.scatter(pca_result[:, 0], pca_result[:, 1], c=[colores[barrio] for barrio in y_district], alpha=0.7)

# Agregar información al hacer clic en los puntos
cursor = mplcursors.cursor(scatter, hover=True)

def on_point_hover(sel):
    index = sel.target.index
    district_info = y_district[index]
    sel.annotation.set_text(f"Distrito: {district_info[0]}\nConcepto: {district_info[1]}")

cursor.connect("add", on_point_hover)

ax.set_xlabel('Componente Principal 1')
ax.set_ylabel('Componente Principal 2')
ax.set_title('Gráfico de Componentes Principales')
ax.grid(True)

plt.show()